# *Applied Data Science Capstone*

## The Battle of the Neighborhoods - Week 1 (Course 9 Week 4 Assignment)

### 1. Problem and Discussion of the Background

While it is widely known that New York is the financial centre of the world, people sometimes fail to realize that there is more than one financial centre within the United States (US) itself, let alone internationally. I'm interested in assessing how similar (or diverse for that matter) different financial capitals are. For this reason, I'd like to compare the Manhattan, NY to Los Angeles, CA (another important financial centre for the US) to see if the makeup of those areas show similar trends or do other factors take precedence and explain the dissimilarities. Using the clustering analysis, I will compare the results to make inferences about the areas. I hope to learn more about the areas even though I have visited both places numerous times. 

### 2. Data Needed and Solution Outline

Based on what we've learned throughout this program, the data needed for my discussion revolves around collecting geospatial data along with some descriptive information (i.e. venue information) linked to the geospatial data. FourSquare is a great platform to collect such information as it provides coordinates as well which will make mapping easier. Additionally, I'll need to scrape "neighborhood" information from web sources that list the different ZIP codes for both Manhattan, NY and Los Angeles, CA. Once the data is collected, I'll have to clean it and extract only the data that will be pertinent to my discussion. Once, I complete the data cleaning process, I will move on to the analysis stage where I map and cluster the data using k-Means clustering. Finally, using the information computed by the k-Means clustering, I will make my conclusions about the aforementioned locations.